In [ ]:
import math
import pandas as pd
import altair as alt
import numpy as np
from src.spinorama.analysis import sound_power
from src.spinorama.load import parse_all_speakers, parse_graphs_speaker, graph_melt
from src.spinorama.graph import graph_params_default

# df = parse_graphs_speaker('Adam', 'Adam S2V', 'klippel')
# df = parse_graphs_speaker('Neumann', 'Neumann KH 80', 'klippel')
df = parse_graphs_speaker('Genelec', 'Genelec 8030A', 'princeton')
# print(df)

In [ ]:
def estimates(onaxis: pd.DataFrame):
    try:
        freq_min = onaxis.Freq.min()
        if freq_min < 300:
            # mean over 300-10k                                                                                                                       
            y_ref = np.mean(onaxis.loc[(onaxis.Freq>=300) & (onaxis.Freq<=10000)].dB)
            y_3 = onaxis.loc[(onaxis.Freq<150)&(onaxis.dB<=y_ref-3)].Freq.max()
            y_6 = onaxis.loc[(onaxis.Freq<150)&(onaxis.dB<=y_ref-6)].Freq.max()
            # search band up/down                                                                                                                     
            up:   float = onaxis.loc[(onaxis.Freq>=100) & (onaxis.Freq<=10000)].dB.max()
            down: float = onaxis.loc[(onaxis.Freq>=100) & (onaxis.Freq<=10000)].dB.min()
            band = max(abs(up-y_ref), abs(y_ref-down))
            return [round(y_ref, 0), round(y_3, 0) , round(y_6, 0), round(band,0)]
        else:
            y_ref = np.mean(onaxis.loc[(onaxis.Freq>=freq_min) & (onaxis.Freq<=10000)].dB)
            # search band up/down                                                                                                                     
            up:   float = onaxis.loc[(onaxis.Freq>=freq_min) & (onaxis.Freq<=10000)].dB.max()
            down: float = onaxis.loc[(onaxis.Freq>=freq_min) & (onaxis.Freq<=10000)].dB.min()
            band = max(abs(up-y_ref), abs(y_ref-down))
            return [round(y_ref, 0), -1, -1, round(band,0)]
    except TypeError as te:
        logging.warning('Estimates failed for {0} with {1}'.format(onaxis.shape, te))
        return [-1, -1, -1, -1]
    except ValueError as ve:
        logging.warning('Estimates failed for {0} with {1}'.format(onaxis.shape, ve))
        return [-1, -1, -1, -1]

In [ ]:
spin = df['CEA2034']
estimates(spin.loc[spin['Measurements'] == 'On Axis'])
